In [1]:

!pip install tensorboardx
!pip install transformers
!pip install simpletransformers


     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 645kB 2.7MB/s 
     |████████████████████████████████| 890kB 13.1MB/s 
     |████████████████████████████████| 3.8MB 17.2MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=fdd1358b34026269988b19e2fd4338d030a13c850dec4c5d61cdbcbacd188b51
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 194kB 2.6MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=eac3e65d3d3167944e2389620065ddd87b8b86bd67f64564bbda6f007577b10c
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [2]:
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir  ./apex



Cloning into 'apex'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 6734 (delta 0), reused 1 (delta 0), pack-reused 6726
Receiving objects: 100% (6734/6734), 13.74 MiB | 6.84 MiB/s, done.
Resolving deltas: 100% (4492/4492), done.
Created temporary directory: /tmp/pip-ephem-wheel-cache-zhqj0kaj
Created temporary directory: /tmp/pip-req-tracker-ngtq7f7a
Created requirements tracker '/tmp/pip-req-tracker-ngtq7f7a'
Created temporary directory: /tmp/pip-install-278hb4ad
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-1agg3h7c
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-ngtq7f7a'
    Running setup.py (path:/tmp/pip-req-build-1agg3h7c/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.5.0+cu101
    running egg_info
    creating /tmp/pip-req-build-1agg3h7c/pip-egg-info/apex.eg

In [3]:
import pandas as pd
import numpy as np
from transformers import *

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.util import ngrams
stop = stopwords.words('english')
stop.pop(118)

from sklearn.model_selection import train_test_split

from simpletransformers.classification import ClassificationModel

from sklearn.metrics import mean_absolute_error, accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/")
os.listdir()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['pls.ipynb',
 'roberta.ipynb',
 'simple_transformers.ipynb',
 'xlm.ipynb',
 'xlnet.ipynb',
 'distilbert.ipynb',
 'data']

In [0]:
review_path = "data/yelp_review_training_dataset.jsonl"
data = pd.read_json(review_path, lines = True)


In [0]:
data = data.drop(["review_id"], axis =1)

In [17]:
data.head()

,text,stars
0,Total bill for this horrible service? Over $8G...,1
1,I *adore* Travis at the Hard Rock's new Kelly ...,5
2,I have to say that this office really has it t...,5
3,Went in for a lunch. Steak sandwich was delici...,5
4,Today was my second out of three sessions I ha...,1


In [18]:
data["text"] = data['text'].str.replace('[^\w\s]','').str.lower()
data["text"] = data['text'].str.replace('\d+', '')
data['text'] = data['text'].apply(lambda x: [item for item in x.split() if item not in stop])
data['text'] = data['text'].apply(lambda x: ' '.join(x))

data.head()

,text,stars
0,total bill horrible service gs crooks actually...,1
1,adore travis hard rocks new kelly cardenas sal...,5
2,say office really together organized friendly ...,5
3,went lunch steak sandwich delicious caesar sal...,5
4,today second three sessions paid although firs...,1


In [24]:
data.sample(frac=1)
data_sample = data.iloc[:100000]
data_sample.columns = ['text', 'labels']
train, validation = train_test_split(data_sample, test_size=0.2)
data_sample

,text,labels
0,total bill horrible service gs crooks actually...,1
1,adore travis hard rocks new kelly cardenas sal...,5
2,say office really together organized friendly ...,5
3,went lunch steak sandwich delicious caesar sal...,5
4,today second three sessions paid although firs...,1
...,...,...
99995,tom fixed botched acrylicgel job got another s...,5
99996,dustin spivey provided amazing customer servic...,5
99997,super bon belle présentation mes préférés le c...,5
99998,ive literally eaten everything menu love place...,5


In [0]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=6, use_cuda=True, args = {'output_dir' : "outputs_roberta/", "overwrite_output_dir" : True, "train_batch_size":16, "eval_batch_size":16})

In [30]:
model.train_model(train) 



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.749053

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 1.135253

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.860745Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 1.372582Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.931215Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.580916

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.979263Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.542030



In [0]:





apex==0.1
cmake==3.12.0
cupy-cuda101==6.5.0
dataclasses==0.7
en-core-web-sm==2.2.5
GDAL==2.2.2
ideep4py==2.0.0.post3
intel-openmp==2020.0.133
pycocotools==2.0.0
pygobject==3.26.1
python-apt==1.6.5+ubuntu0.2
rpy2==3.2.7
screen-resolution-extra==0.0.0
tbb==2020.0.133
torch==1.5.0+cu101
torchvision==0.6.0+cu101
xkit==0.0.0

In [38]:
result, model_outputs, wrong_predictions = model.eval_model(validation, avg_star_err=mean_absolute_error, exact_match=accuracy_score)

In [32]:
print(result)

{'mcc': 0.6520110973025963, 'avg_star_err': 0.2912, 'exact_match': 0.76775, 'eval_loss': 0.5831386166214942}


In [35]:
predictions, raw_outputs = model.predict(["this restaurant is not good"])

In [36]:
print(predictions[0])
print(raw_outputs)

1
[[-3.8730469   4.5703125   2.9003906   0.81884766 -1.6621094  -2.3652344 ]]


In [0]:
!pip freeze > requirements.txt

Python 3.6.9
